<a href="https://colab.research.google.com/github/jremeh/NLP/blob/master/fastai2_beginner_tabular_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install fastai2

     |████████████████████████████████| 184kB 2.7MB/s 


In [3]:
from fastai2.tabular.all import *

In [5]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/models')]

In [6]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


Continuous columns (like age) will be treated as float numbers to feed the model directly.

Categorical columns will be converted to unique index that will be fed to embedding layers.

We can specify our categorical and continuous column names, as well as the name of the dependent variable in `TabularDataLoaders` factory methods:

In [7]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

The last part is the list of pre-processors we apply to our data:

- `Categorify` is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
- `FillMissing` will fille the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
- `Normalize` will normalize the continuous variables (substract the mean and divide by the std)

In [8]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,32.000000,316588.994679,13.0,>=50k
1,Private,Some-college,Divorced,Priv-house-serv,Other-relative,White,False,68.000001,144136.999061,10.0,<50k
2,Self-emp-not-inc,Some-college,Never-married,Craft-repair,Not-in-family,White,False,34.000000,156809.000616,10.0,<50k
3,Private,Assoc-voc,Married-civ-spouse,Prof-specialty,Wife,White,False,28.000000,247819.000921,11.0,<50k
4,Private,9th,Divorced,Adm-clerical,Unmarried,White,False,36.000000,732569.013177,5.0,<50k
5,Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,White,False,41.000000,173681.999544,11.0,>=50k
6,Private,Some-college,Never-married,Craft-repair,Not-in-family,Black,False,34.000000,405385.994396,10.0,<50k
7,Private,Some-college,Never-married,Adm-clerical,Own-child,White,False,20.000001,103276.999986,10.0,<50k
8,Self-emp-not-inc,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,False,68.000001,69249.002990,9.0,>=50k
9,Self-emp-not-inc,Doctorate,Never-married,Prof-specialty,Not-in-family,White,False,36.000000,343720.998367,16.0,>=50k


**Note**: Sometimes with tabular data, your y's may be encoded (such as 0 and 1). In such a case you should explicitly pass `y_block = CategoryBlock` in your constructor so fastai won't presume you are doing regression.

In [9]:
learn = tabular_learner(dls, metrics=accuracy)

`fine_tune` method won't be useful as we're not using pretrained model

In [10]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.365220,0.348718,0.842291,00:08


/usr/local/lib/python3.6/dist-packages/fastai2/callback/schedule.py:68: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx = (pos >= pcts).nonzero().max()


In [11]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,12.0,3.0,13.0,1.0,5.0,1.0,-0.484704,0.145714,-0.423777,0.0,0.0
1,5.0,16.0,1.0,2.0,2.0,5.0,1.0,0.028017,-0.239045,-0.032213,0.0,0.0
2,5.0,16.0,5.0,9.0,3.0,5.0,1.0,-0.704442,0.021675,-0.032213,0.0,0.0
3,7.0,12.0,5.0,13.0,3.0,2.0,1.0,-0.924180,-1.147583,-0.423777,1.0,0.0
4,5.0,16.0,5.0,13.0,4.0,5.0,1.0,-1.290410,-0.439000,-0.032213,0.0,0.0
5,5.0,10.0,3.0,2.0,1.0,2.0,1.0,0.321001,-0.607177,1.142480,0.0,0.0
6,5.0,8.0,5.0,2.0,5.0,3.0,1.0,-0.191721,-0.672702,0.750915,0.0,0.0
7,5.0,2.0,5.0,13.0,4.0,5.0,1.0,-1.363656,0.440995,-1.206906,0.0,0.0
8,5.0,10.0,3.0,14.0,1.0,5.0,1.0,-0.484704,-0.133782,1.142480,1.0,1.0


In [12]:
learn.predict(df.iloc[0])

(   workclass  education  marital-status  ...    fnlwgt  education-num  salary
 0        5.0        8.0             3.0  ... -0.839156       0.750915     1.0
 
 [1 rows x 11 columns], tensor(1), tensor([0.4132, 0.5868]))

Prediction on a new data frame

In [13]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [16]:
learn.get_preds(dl=dl)

(tensor([[0.4132, 0.5868],
         [0.4597, 0.5403],
         [0.9883, 0.0117],
         ...,
         [0.6110, 0.3890],
         [0.6088, 0.3912],
         [0.6777, 0.3223]]), None)